<a href="https://colab.research.google.com/github/sou-venir/Dream-nori/blob/main/%EC%BD%94%EB%9E%A9%EC%9D%84%EC%9C%84%ED%95%9C%EB%93%9C%EB%A6%BC%EB%86%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# [1] 필수 라이브러리 설치 (코랩 환경 전용)
!pip install flask-socketio flask-ngrok pyngrok openai python-socketio
!fuser -k 5000/tcp # 기존에 사용 중인 포트가 있다면 강제 종료

import os
import json
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit
from pyngrok import ngrok
from google.colab import userdata
import openai
from google.colab import drive
import openai
import google.generativeai as genai

# [2] 구글 드라이브 마운트 (채팅 로그 및 설정값 영구 저장)
drive.mount('/content/drive')

# 저장 경로 설정: 내 드라이브의 'ChatData' 폴더
SAVE_PATH = '/content/drive/MyDrive/ChatData'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# [3] 보안 및 AI 설정
# 코랩 왼쪽 '열쇠' 아이콘 메뉴에서 NGROK_AUTH_TOKEN과 OPENAI_API_KEY를 추가해야 합니다! 발급받은 키를 설정해 주세요
try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-3-pro-preview')
    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print(f"❌ 시크릿 설정 확인 필요: {e}")

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

# [수정] 이 부분을 app = Flask(__name__) 바로 아래에 추가해줘!
@app.route('/')
def index():
    # HTML_TEMPLATE을 화면에 렌더링해서 보여주는 역할
    return render_template_string(HTML_TEMPLATE, theme=state.get('theme'))

DATA_FILE = os.path.join(SAVE_PATH, "save_data.json")

# [저장 로직] 상태가 변할 때마다 구글 드라이브에 JSON 파일로 기록
def save_data():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=4)

# [불러오기 로직] 서버 시작 시 기존 저장된 데이터를 가져옴
def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return None
    return None

# [4] 서버 상태 관리 (기본값 설정)
initial_state = {
    "session_title": "드림놀이",
    "theme": {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"},
    "accent_color": "#e91e63",
    "ai_model": "gpt-5.2", # 기본값 설정
    "admin_password": "3896",
    "is_locked": False,
    "inputs": {"user1": None, "user2": None},
    "profiles": {
        "user1": {"name": "Player 1", "bio": "", "canon": ""},
        "user2": {"name": "Player 2", "bio": "", "canon": ""}
    },
    "history": [],
    "ai_history": [],
    "summary": "기록된 줄거리가 없습니다.",
    "prologue": "이곳에 프롤로그를 입력하세요.",
    "sys_prompt": "당신은 숙련된 TRPG 마스터입니다.",
    "lorebook": []
}

saved_state = load_data()
state = saved_state if saved_state else initial_state

# [5] AI 테마 분석 로직
def analyze_theme_color(title, sys_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "system",
                "content": "너는 웹 디자인 전문가야. 세션 설정에 어울리는 테마 색상을 골라줘. 모든 UI 텍스트는 검은색이야. JSON 형식으로 답변해: {\"bg\": \"배경색\", \"panel\": \"패널색\", \"accent\": \"강조색\"}"
            }, {
                "role": "user",
                "content": f"제목: {title}\n설정: {sys_prompt}"
            }],
            response_format={ "type": "json_object" }
        )
        palette = json.loads(response.choices[0].message.content)
        default = {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}
        default.update(palette)
        return default
    except:
        return {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}

# [6] 프론트엔드 HTML 템플릿
HTML_TEMPLATE = """<!DOCTYPE html>

<html>

<head>

    <meta charset=\"UTF-8\">

    <title>드림놀이</title>

    <script src=\"https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js\"></script>

    <script src=\"https://cdn.jsdelivr.net/npm/marked/marked.min.js\"></script>

    <style>

        :root {

            --bg: {{ theme.bg if theme else '#ffffff' }};

            --panel: {{ theme.panel if theme else '#f1f3f5' }};

            --accent: {{ theme.accent if theme else '#e91e63' }};

            --text: #000000;

        }


        /* 1. 모달이 화면을 벗어나지 않게 고정 */

        html, body { height: 100%; margin: 0; overflow: hidden; }

        body { font-family: 'Pretendard', sans-serif; display: flex; background: var(--bg); color: #000000 !important; }

        div, p, span, h1, h2, h3, h4, input, textarea, select, button, .bubble { color: #000000 !important; }



        #main { flex: 1; display: flex; flex-direction: column; height: 100vh; border-right: 1px solid rgba(0,0,0,0.05); }

        #chat-window { flex: 1; overflow-y: auto; padding: 30px 10%; display: flex; flex-direction: column; gap: 15px; }

        #sidebar { width: 320px; height: 100vh; background: var(--panel); padding: 20px; box-sizing: border-box; overflow-y: auto; display: flex; flex-direction: column; gap: 12px; }



        textarea, input, select {

            background: var(--bg) !important;

            border: 1px solid rgba(0, 0, 0, 0.1) !important;

            border-radius: 10px; padding: 10px; width: 100%; box-sizing: border-box;

            transition: all 0.2s ease; resize: none !important;

        }

        #msg-input { background: var(--panel) !important; border: 1px solid rgba(0, 0, 0, 0.15) !important; height: 80px; }

        textarea:focus, input:focus { outline: none; border-color: var(--accent) !important; box-shadow: 0 0 5px rgba(0,0,0,0.05); }



        .bubble { padding: 15px 20px; border-radius: 15px; max-width: 85%; line-height: 1.6; font-size: 14px; white-space: pre-wrap; background: rgba(0,0,0,0.03); }

        .center-ai { align-self: center; background: var(--panel) !important; border-left: 5px solid var(--accent); width: 100%; max-width: 800px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); }

        .user-bubble { align-self: flex-end; border-right: 5px solid var(--accent); background: var(--bg); }



        button { cursor: pointer; border: none; border-radius: 8px; background: var(--accent); padding: 10px; font-weight: bold; transition: 0.2s; }

        button:hover { opacity: 0.8; }

        .btn-reset { background: #ff4444 !important; color: #ffffff !important; margin-top: 20px; }



        /* 1. 모달 배경 및 컨테이너 */
#admin-modal {
    display: none;
    position: fixed;
    z-index: 10000;
    left: 0; top: 0;
    width: 100vw; height: 100vh;
    background: rgba(0, 0, 0, 0.6);
    backdrop-filter: blur(5px);
    align-items: center; justify-content: center;
}

.modal-content {
    width: 95%; max-width: 1200px; height: 85vh;
    background: #ffffff; border-radius: 16px;
    display: flex; flex-direction: column;
    box-shadow: 0 20px 60px rgba(0,0,0,0.3);
    overflow: hidden;
}

/* 2. 상단 헤더 & 탭 메뉴 */
.modal-header {
    height: 60px; display: flex; justify-content: space-between; align-items: center;
    padding: 0 25px; background: #f8f9fa; border-bottom: 1px solid #eee;
}

.tab-group { display: flex; height: 100%; gap: 10px; }
.tab-btn {
    border: none; background: none; padding: 0 15px;
    font-size: 14px; font-weight: 600; color: #777;
    cursor: pointer; position: relative; transition: 0.2s;
}
.tab-btn.active { color: var(--accent); }
.tab-btn.active::after {
    content: ""; position: absolute; bottom: 0; left: 0;
    width: 100%; height: 3px; background: var(--accent);
}

.close-btn {
    width: 32px; height: 32px; border-radius: 50%; border: none;
    background: #eee; cursor: pointer; font-size: 16px;
}

/* 3. 모달 바디 (좌우 분할) */
.modal-body { flex: 1; display: flex; overflow: hidden; }

.tab-content {
    display: none; width: 100%; height: 100%;
    flex-direction: row; /* 좌우 배치 */
}
.tab-content.active { display: flex; }

/* 왼쪽 편집창 */
.editor-side {
    flex: 1.3; padding: 25px; display: flex; flex-direction: column;
    gap: 15px; overflow-y: auto; border-right: 1px solid #f0f0f0;
}

/* 오른쪽 정보창 */
.list-side {
    flex: 0.7; padding: 25px; background: #fafafa;
    display: flex; flex-direction: column; gap: 15px; overflow-y: auto;
}

/* 4. 내부 요소 디자인 */
.editor-side label, .list-side label {
    font-size: 12px; font-weight: 800; color: #999; text-transform: uppercase;
}

.editor-side input, .editor-side select, .editor-side textarea, .list-side textarea {
    width: 100%; border: 1px solid #ddd; border-radius: 8px;
    padding: 12px; font-size: 14px; font-family: inherit;
    background: #fff !important;
}

.editor-side textarea { flex: 1; min-height: 200px; resize: none; }
.list-side textarea { height: 100%; resize: none; }

.save-btn {
    background: var(--accent); color: white !important;
    padding: 15px; border-radius: 10px; font-weight: bold;
    cursor: pointer; border: none; margin-top: 5px;
}

/* 키워드 아이템 */
.lore-item {
    background: #fff; border: 1px solid #eee; padding: 12px;
    border-radius: 10px; position: relative; margin-bottom: 8px;
}

    </style>

</head>

<body>

    <div id=\"main\">

        <div id=\"chat-window\"><div id=\"chat-content\"></div></div>

        <div id=\"input-area\" style=\"padding:20px; background: var(--bg);\">

            <div id=\"status\" style=\"font-size: 12px; margin-bottom: 5px; color: var(--accent); font-weight: bold;\">대기 중</div>

            <div style=\"display:flex; gap:10px;\">

                <textarea id=\"msg-input\" placeholder=\"설정 완료 후 잠금 버튼을 눌러주세요.\"></textarea>

                <button onclick=\"send()\" style=\"width:80px;\">전송</button>

            </div>

        </div>

    </div>



   <div id="sidebar">



        <h3>🎭 설정</h3>



        <select id="user-role" onchange="refreshUI()">



            <option value="user1">Player 1</option>



            <option value="user2">Player 2</option>



        </select>



        <input type="text" id="p-name" placeholder="이름">
    <textarea id="p-bio" style="height:120px;" placeholder="캐릭터 설정"></textarea>
    <textarea id="p-canon" style="height:80px;" placeholder="관계 설정"></textarea>

    <button onclick="saveProfile()" id="ready-btn" style="background:var(--accent); color:white !important;">
        ✅ 설정 저장 및 준비 완료
    </button>

    <div id="ready-status" style="font-size:11px; margin-top:5px; color:#666;">
        대기 중...
    </div>

    <div style="flex: 1;"></div>
    <button onclick="requestAdmin()" style="background:transparent; color:#999 !important; border: 1px solid #ddd;">⚙️ 마스터 설정 </button>
</div>



    <div id="admin-modal">
        <div class="modal-content">
            <div class="modal-header">
                <div class="tab-group">
                    <button class="tab-btn active" onclick="openTab(event, 't-base')">⚙️ 엔진</button>
                    <button class="tab-btn" onclick="openTab(event, 't-story')">🎬 서사</button>
                    <button class="tab-btn" onclick="openTab(event, 't-ex')">💡 학습</button>
                    <button class="tab-btn" onclick="openTab(event, 't-lore')">📚 키워드</button>
                </div>
                <button onclick="closeModal()" class="close-btn">✕</button>
            </div>

            <div class="modal-body">
                <div id="t-base" class="tab-content active">
                    <div class="editor-side">
                        <label>AI 모델 선택</label>
                        <select id="m-ai-model">
                            <option value="gpt-5.2">OpenAI GPT-5.2</option>
                            <option value="gpt-4o">OpenAI GPT-4o</option>
                            <option value="gemini-3-pro-preview">Google Gemini 3 Pro</option>
                        </select>
                        <label>시스템 프롬프트 (AI 지침)</label>
                        <textarea id="m-sys" placeholder="AI에게 줄 지침..."></textarea>
                        <button onclick="saveMaster()" class="save-btn">💾 엔진 설정 저장</button>
                    </div>
                    <div class="list-side">
                        <label>안내</label>
                        <p style="font-size:13px; color:#666;">엔진 모델과 전체적인 AI의 페르소나를 결정합니다.</p>
                        <button class="btn-reset" onclick="sessionReset()" style="margin-top: auto;">⚠️ 세션 완전 초기화</button>
                    </div>
                </div>

                <div id="t-story" class="tab-content">
                    <div class="editor-side">
                        <label>🏷️ 세션 제목</label>
                        <input type="text" id="m-title" placeholder="제목">

                        <label>📌 현재 상황 요약</label>
                        <textarea id="m-sum" style="height:100px; flex:none;" placeholder="지금까지의 핵심 내용..."></textarea>

                        <label>📖 프롤로그</label>
                        <textarea id="m-pro" placeholder="이야기의 시작..."></textarea>

                        <button onclick="saveMaster()" class="save-btn">💾 모든 서사 저장</button>
                    </div>
                    <div class="list-side">
                        <label>💡 서사 팁</label>
                        <p style="font-size:13px; color:#666;">서사는 AI가 이야기의 맥락을 파악하는 데 가장 중요한 정보야.</p>
                    </div>
                </div>

                <div id="t-ex" class="tab-content">
                    <div class="editor-side">
                        <label>💡 학습 데이터 (대화 예시)</label>
                        <textarea id="ex-data" placeholder="[User]: 안녕!&#10;[AI]: 반가워요!"></textarea>
                        <button onclick="saveExamples()" class="save-btn">💡 학습 데이터 저장</button>
                    </div>
                    <div class="list-side"><label>도움말</label><p style="font-size:12px;">원하는 말투를 직접 적어줘.</p></div>
                </div>

                <div id="t-lore" class="tab-content">
                    <div class="editor-side">
                        <label>🔍 키워드 이름</label>
                        <input type="text" id="kw-t" placeholder="이름">
                        <label>🎯 트리거 (단어 입력 후 엔터/스페이스)</label>
                        <div id="tag-container">
                            <input type="text" id="tag-input" placeholder="태그 추가..." style="border:none !important; width: 100px !important; outline:none; background:transparent !important;">
                        </div>
                        <label>📝 상세 설정</label>
                        <textarea id="kw-c" placeholder="AI에게 전달할 설정 내용..."></textarea>
                        <button id="lore-save-btn" onclick="addLoreWithTags()" class="save-btn">➕ 키워드 저장</button>
                    </div>
                    <div class="list-side">
                        <label>📋 우선순위 (드래그하여 이동)</label>
                        <div id="lore-list" style="flex: 1; overflow-y: auto; display:flex; flex-direction:column; gap:8px;"></div>
                    </div>
                </div>
            </div>
        </div>
    </div>
<script>

    const socket = io();

    let gState = null;



    // [수정] 상태 알림 리스너 추가 (AI 응답 중, 저장 완료 등 표시)

    socket.on('status_update', d => {

        const statusEl = document.getElementById('status');

        if(statusEl) {

            statusEl.innerText = d.msg;

            if(d.msg.includes('❌')) statusEl.style.color = 'red';

            else statusEl.style.color = 'var(--accent)';

        }

    });



    socket.on('initial_state', data => {

        gState = data;

        if (data.theme) {

            const root = document.documentElement.style;

            root.setProperty('--bg', data.theme.bg);

            root.setProperty('--panel', data.theme.panel);

            root.setProperty('--accent', data.theme.accent);

        }

        refreshUI();

    });

    // 타이핑 효과 함수
function typeWriter(element, text, i = 0) {
    if (i === 0) {
        element.innerHTML = ""; // 처음 시작할 때 비우기
        element.style.whiteSpace = "pre-wrap"; // 줄바꿈 유지
    }

    if (i < text.length) {
        // 텍스트를 한 글자씩 추가 (마크다운 적용 전 raw 텍스트로)
        element.textContent += text.charAt(i);
        i++;

        // 스크롤 아래로 고정
        const win = document.getElementById('chat-window');
        win.scrollTop = win.scrollHeight;

        setTimeout(() => typeWriter(element, text, i), 35); // 35ms 속도로 출력
    } else {
        // 타이핑이 모두 끝나면 최종적으로 마크다운 렌더링 적용
        element.innerHTML = marked.parse(text);
    }
}

    function refreshUI() {

        if(!gState) return;

        renderChat();

        renderLore();

        applyLockUI();

    function applyLockUI() {
    if(!gState) return;

    const role = document.getElementById('user-role').value;
    const p = gState.profiles[role];

    // 이미 이름이 저장되어 있는 상태라면 (즉, 한번 확정했다면)
    if(p.name && p.name !== "Player 1" && p.name !== "Player 2") {
        document.getElementById('p-name').readOnly = true;
        document.getElementById('p-bio').readOnly = true;
        document.getElementById('p-canon').readOnly = true;
        document.getElementById('ready-btn').disabled = true;
        document.getElementById('ready-btn').innerText = "🔒 설정 고정됨";
    } else {
        // 아직 설정 전이라면 풀어주기
        document.getElementById('p-name').readOnly = false;
        document.getElementById('p-bio').readOnly = false;
        document.getElementById('p-canon').readOnly = false;
        document.getElementById('ready-btn').disabled = false;
        document.getElementById('ready-btn').innerText = "✅ 설정 저장 및 준비 완료";
    }
}

        const role = document.getElementById('user-role').value;

        const p = gState.profiles[role];



        // [수정] 현재 포커스 된 입력창은 덮어쓰지 않음 (타이핑 방해 금지)

        const activeId = document.activeElement.id;



        if(activeId !== 'p-name') document.getElementById('p-name').value = p.name || "";

        if(activeId !== 'p-bio') document.getElementById('p-bio').value = p.bio || "";

        if(activeId !== 'p-canon') document.getElementById('p-canon').value = p.canon || "";



        if(activeId !== 'm-title') document.getElementById('m-title').value = gState.session_title || "";

        if(activeId !== 'm-sys') document.getElementById('m-sys').value = gState.sys_prompt || "";

        if(activeId !== 'm-pro') document.getElementById('m-pro').value = gState.prologue || "";

        if(activeId !== 'm-sum') document.getElementById('m-sum').value = gState.summary || "";



        for(let i=0; i<3; i++) {

            if(gState.examples && gState.examples[i]) {

                if(activeId !== `ex-q-${i}`) document.getElementById(`ex-q-${i}`).value = gState.examples[i].q || "";

                if(activeId !== `ex-a-${i}`) document.getElementById(`ex-a-${i}`).value = gState.examples[i].a || "";

            }

        }

    }



        function renderChat() {
    let h = `<div style="text-align:center; padding:20px; color:var(--accent); font-weight:bold; font-size:1.4em;">${gState.session_title}</div>`;
    h += `<div class="bubble center-ai"><b>[PROLOGUE]</b><br>${marked.parse(gState.prologue || "")}</div>`;

    const contentDiv = document.getElementById('chat-content');
    const history = gState.ai_history;
    const role = document.getElementById('user-role').value;
    const pName = gState.profiles[role].name;

    // 전체 히스토리 렌더링
    history.forEach((msg, index) => {
        const isUser = pName && msg.includes(`**${pName}**:`);
        const isLastMsg = (index === history.length - 1);
        const isAI = msg.startsWith("**AI**:");

        // 마지막 메시지가 AI인 경우에만 타이핑 효과 적용
        if (isLastMsg && isAI) {
            const bubbleId = `typing-${index}`;
            h += `<div id="${bubbleId}" class="bubble center-ai"></div>`;
            contentDiv.innerHTML = h; // 먼저 틀을 만들고

            const targetElement = document.getElementById(bubbleId);
            typeWriter(targetElement, msg); // 타이핑 시작!
        } else {
            h += `<div class="bubble ${isUser ? 'user-bubble' : 'center-ai'}">${marked.parse(msg)}</div>`;
        }
    });

    if (history.length === 0 || !history[history.length-1].startsWith("**AI**:")) {
        contentDiv.innerHTML = h;
    }

    const win = document.getElementById('chat-window');
    win.scrollTop = win.scrollHeight;
}


        function send() {
    const input = document.getElementById('msg-input');
    const text = input.value.trim();
    // gState.is_locked 체크를 없애거나, 저장 시 True가 되게 해야 함
    if(!text) return;
    socket.emit('client_message', { uid: document.getElementById('user-role').value, text });
    input.value = '';
}



        function confirmLock() {

            if(confirm("설정을 완료하고 채팅을 시작하시겠습니까?")) {

                socket.emit('lock_settings');

            }

        }



        function requestAdmin() {

            const pw = prompt("관리자 비밀번호:");

            if(pw) socket.emit('check_admin', { password: pw });

        }



       socket.on('admin_auth_res', d => {
    if(d.success) {
        // 1. 모달 띄우기
        const modal = document.getElementById('admin-modal');
        modal.style.display = 'flex';

        // 2. 모든 탭 숨기기 및 버튼 비활성화 (초기화)
        document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));
        document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active'));

        // 3. 첫 번째 탭(시스템)만 강제로 켜기
        document.getElementById('t-base').classList.add('active');
        document.querySelector('.tab-btn').classList.add('active');

        refreshUI(); // 저장된 데이터 다시 불러와서 칸 채우기
    } else {
        alert("비밀번호 불일치");
    }
});



        function saveMaster() {
    // 마스터 창에 있는 모든 입력값을 긁어모아!
    const masterData = {
        title: document.getElementById('m-title').value,
        sys: document.getElementById('m-sys').value,
        pro: document.getElementById('m-pro').value,
        sum: document.getElementById('m-sum').value,
        model: document.getElementById('m-ai-model').value // 마스터 창의 엔진 선택값
    };

    socket.emit('save_master_base', masterData);
    alert("마스터 설정이 모두 저장되었어! 엔진이 " + masterData.model + "(으)로 교체됐어.");
    closeModal();
}



        function saveExamples() {

            const exs = [];

            for(let i=0; i<3; i++) {

                exs.push({

                    q: document.getElementById(`ex-q-${i}`).value,

                    a: document.getElementById(`ex-a-${i}`).value

                });

            }

            socket.emit('save_examples', exs);

            alert("AI 학습 데이터 저장 완료.");

        }



        function addLoreWithTags() {
        // HTML의 ID(kw-t, tag-input, kw-c)를 정확하게 가져오도록 수정했어
        const title = document.getElementById('kw-t').value;
        const tags = document.getElementById('tag-input').value;
        const content = document.getElementById('kw-c').value;

        // 제목이나 내용이 비어있으면 경고
        if(!title) return alert("키워드 이름을 입력해줘!");
        if(!content) return alert("설정 내용을 입력해줘!");

        // 서버로 전송
        socket.emit('add_lore', {
            title: title,
            triggers: tags, // 태그(트리거)도 같이 보냄
            content: content,
            priority: 0 // 우선순위 입력창이 없으니 기본값 0으로 설정
        });

        // 입력창 깨끗하게 비우기
        document.getElementById('kw-t').value = "";
        document.getElementById('tag-input').value = "";
        document.getElementById('kw-c').value = "";

        // (선택사항) 저장됐다고 알려주기 싫으면 이 줄은 지워도 돼
        // alert("키워드가 저장됐어!");
    }

        function editLore(idx) {

            const l = gState.lorebook[idx];

            document.getElementById('kw-t').value = l.title;

            document.getElementById('kw-tr').value = l.triggers || "";

            document.getElementById('kw-c').value = l.content;

            document.getElementById('kw-p').value = l.priority || 0;

            if(confirm("수정 모드: 이 키워드를 삭제하고 입력창으로 불러올까요?")) {

                socket.emit('del_lore', { index: idx });

            }

        }



       function renderLore() {
    const listDiv = document.getElementById('lore-list');
    if(!gState || !gState.lorebook) return;

    listDiv.innerHTML = gState.lorebook.map((l, i) => `
        <div style="padding:8px; background:rgba(0,0,0,0.03); margin-bottom:5px; border-radius:8px; display:flex; justify-content:space-between; align-items:center; border: 1px solid rgba(0,0,0,0.05);">
            <span onclick="editLore(${i})" style="cursor:pointer; flex:1; font-size:13px;">
                <b>${l.title}</b> <small style="color:#666;">(우선순위: ${l.priority})</small>
            </span>

            <div style="display:flex; gap:3px;">
                <button onclick="editLore(${i})" style="padding:2px 8px; font-size:11px; background:#44aaff; color:white !important;">수정</button>
                <button onclick="socket.emit('del_lore', {index:${i}})" style="padding:2px 8px; font-size:11px; background:#ff4444; color:white !important;">삭제</button>
            </div>
        </div>`).join('');
}


        function openTab(evt, id) {

            document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));

            document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active'));

            document.getElementById(id).classList.add('active');

            evt.currentTarget.classList.add('active');

        }



        function closeModal() { document.getElementById('admin-modal').style.display='none'; }

        function saveProfile() {
    const role = document.getElementById('user-role').value;
    const name = document.getElementById('p-name').value;

    if(!name || name.includes("Player")) {
        return alert("캐릭터 이름을 먼저 입력해주세요!");
    }

    // [핵심] "못 바꿉니다" 경고창
    const logic = `⚠️ 주의: 지금 설정한 내용으로 확정됩니다.\n세션이 시작된 후에는 내용을 수정할 수 없습니다.\n\n정말로 저장하시겠습니까?`;

    if(confirm(logic)) {
        const data = {
            uid: role,
            name: name,
            bio: document.getElementById('p-bio').value,
            canon: document.getElementById('p-canon').value
        };

        socket.emit('update_profile', data);

        // 저장 후 입력창들 잠그기 (AI 혼란 방지)
        document.getElementById('p-name').readOnly = true;
        document.getElementById('p-bio').readOnly = true;
        document.getElementById('p-canon').readOnly = true;
        document.getElementById('ready-btn').disabled = true;
        document.getElementById('ready-btn').innerText = "🔒 설정 고정됨";

        alert("설정이 고정되었습니다. 상대방의 준비를 기다립니다.");
    }
}

        function sessionReset() { if(confirm("전체 초기화하시겠습니까?")) { const pw = prompt("관리자 비밀번호:"); if(pw) socket.emit('reset_session', { password: pw }); } }



        document.getElementById('msg-input').addEventListener('keydown', e => { if(e.key==='Enter' && !e.shiftKey) { e.preventDefault(); send(); } });

        socket.emit('request_data');

    </script>

</body>

</html>
"""

# [7] 서버 측 소켓 통신 핸들러 (수정됨)

# 헬퍼 함수: 상태 저장 및 전체 전파
def broadcast_state(msg=None):
    save_data()
    emit('initial_state', state, broadcast=True)
    if msg:
        emit('status_update', {'msg': msg}, broadcast=True)

@socketio.on('request_data')
def handle_request():
    emit('initial_state', state)

@socketio.on('lock_settings')
def on_lock_settings():
    p1 = state["profiles"].get("user1", {})
    p2 = state["profiles"].get("user2", {})
    if not p1.get("name") or not p2.get("name"):
        emit('status_update', {'msg': '❌ 양쪽 이름이 모두 있어야 시작 가능합니다.'})
        return
    state["is_locked"] = True
    broadcast_state('🔒 설정이 고정되었습니다.')

@socketio.on('client_message')
def on_client_message(data):
    user_text = data.get('text', '').strip()
    uid = data.get('uid')
    if not user_text: return

    # 1. 키워드 필터링
    active_context = [
        f"[{l['title']}]: {l['content']}"
        for l in state.get('lorebook', [])
        if any(t.strip() in user_text for t in l.get('triggers', '').split(','))
    ]

    # 2. 프롬프트 구성
    messages = [{"role": "system", "content": f"{state['sys_prompt']}\n\n[줄거리]: {state['summary']}\n[참고]: {' '.join(active_context[:3])}"}]

    # 예시 대화(학습 데이터) 추가 [누락된 기능 연결]
    if state.get('examples'):
        for ex in state['examples']:
            if ex.get('q') and ex.get('a'):
                messages.append({"role": "user", "content": ex['q']})
                messages.append({"role": "assistant", "content": ex['a']})

    # 최근 기록 15개 추가
    for h in state['ai_history'][-15:]:
        messages.append({"role": "assistant" if h.startswith("**AI**") else "user", "content": h})

    current_name = state['profiles'].get(uid, {}).get('name', 'User')
    messages.append({"role": "user", "content": f"{current_name}: {user_text}"})

    ai_response = ""

    try:
        current_model = state.get("ai_model", "gpt-5.2")
        emit('status_update', {'msg': f'🤔 {current_model} 엔진 가동 중...'}, broadcast=True)

        # [수정] 모델 이름에 'gemini'가 포함되어 있으면 제미나이 로직 실행
        if "gemini" in current_model.lower():
            full_prompt = f"{state['sys_prompt']}\n\n[줄거리]: {state['summary']}\n"
            full_prompt += "\n".join(state['ai_history'][-10:])
            full_prompt += f"\n{current_name}: {user_text}\nAI:"

            # gemini-3-pro-preview 등 사용자가 설정한 모델명 그대로 호출
            try:
                # 위에서 정의된 gemini_model은 고정값이니, 동적 모델 생성을 시도
                gen_model = genai.GenerativeModel(current_model)
                response = gen_model.generate_content(full_prompt)
            except:
                # 실패하면 기본 gemini_model 사용
                response = gemini_model.generate_content(full_prompt)

            ai_response = response.text
        else:
            # OpenAI 로직
            response = client.chat.completions.create(model=current_model, messages=messages)
            ai_response = response.choices[0].message.content

        state["ai_history"].append(f"**{current_name}**: {user_text}")
        state["ai_history"].append(f"**AI**: {ai_response}")

        broadcast_state('✅ 응답 완료')

    except Exception as e:
        emit('status_update', {'msg': f'❌ 에러: {str(e)}'}, broadcast=True)

@socketio.on('save_master_base')
def on_save_master(data):
    # [수정] 끊겨있던 저장 로직 연결
    state.update({
        "session_title": data['title'],
        "sys_prompt": data['sys'],
        "prologue": data['pro'],
        "summary": data['sum'],
        "ai_model": data.get('model', 'gpt-5.2'),
        "is_locked": True
    })

    # [수정] 죽어있던 테마 분석 함수 부활!
    try:
        new_theme = analyze_theme_color(state["session_title"], state["sys_prompt"])
        state["theme"] = new_theme
    except:
        pass

    broadcast_state("💾 마스터 설정 저장 & 테마 적용 완료")

@socketio.on('save_examples')
def on_save_examples(data):
    # [수정] 누락된 핸들러 추가
    state["examples"] = data
    broadcast_state("💡 학습 데이터 저장 완료")

@socketio.on('update_profile')
def on_profile(data):
    uid = data.get('uid')
    state["profiles"][uid].update({
        "name": data['name'],
        "bio": data['bio'],
        "canon": data['canon']
    })

    p1_name = state["profiles"]["user1"].get("name", "")
    p2_name = state["profiles"]["user2"].get("name", "")

    if p1_name and p1_name != "Player 1" and p2_name and p2_name != "Player 2":
        state["is_locked"] = True
        broadcast_state('🚀 준비 완료! 세션 시작')
    else:
        state["is_locked"] = False
        broadcast_state('⏳ 상대방 대기 중...')

@socketio.on('add_lore')
def on_add_lore(data):
    state.setdefault("lorebook", []).append(data)
    broadcast_state("➕ 키워드 추가됨")

@socketio.on('del_lore')
def on_del_lore(data):
    try:
        state["lorebook"].pop(data['index'])
        broadcast_state("🗑️ 키워드 삭제됨")
    except: pass

@socketio.on('check_admin')
def check_admin(data):
    emit('admin_auth_res', {'success': str(data.get('password')) == str(state.get('admin_password'))})

@socketio.on('reset_session')
def on_reset_session(data):
    if str(data.get('password')) == str(state.get('admin_password')):
        state.update({"ai_history": [], "lorebook": [], "is_locked": False})
        broadcast_state("♻️ 세션 초기화됨")
    else:
        emit('status_update', {'msg': '❌ 비번 틀림'})
# --- 7. 서버 실행부 (수정됨) ---
if __name__ == '__main__':
    try:
        # 1. 시스템에 남아있는 ngrok 프로세스를 완전히 청소
        import subprocess
        subprocess.run(["pkill", "-9", "ngrok"])


        from pyngrok import ngrok
        ngrok.kill() # pyngrok 내부 터널 종료

        # 2. 터널 생성 (기존 터널이 있으면 삭제 후 생성)
        public_url = ngrok.connect(5000).public_url

        print("\n" + "="*50)
        print(f"🚀 드림놀이 서버 실행 중!")
        print(f"🔗 접속 주소: {public_url}")
        print(f"🔐 마스터 암호: {state.get('admin_password', '3896')}")
        print("="*50 + "\n")

        # 3. 서버 실행
        socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

    except Exception as e:
        print(f"❌ 서버 실행 중 오류 발생: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.9 MB/s eta 0:00:00
Mounted at /content/drive



🚀 드림놀이 서버 실행 중!
🔗 접속 주소: https://unadhesively-prolix-deb.ngrok-free.dev
🔐 마스터 암호: 3896

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:54] "GET /socket.io/?EIO=4&transport=polling&t=PjAkxDw HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:54] "POST /socket.io/?EIO=4&transport=polling&t=PjAkxM4&sid=zpUW_c3NIDUA5DMjAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:54] "GET /socket.io/?EIO=4&transport=polling&t=PjAkxM7&sid=zpUW_c3NIDUA5DMjAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:55] "GET /socket.io/?EIO=4&transport=polling&t=PjAkxV5.0&sid=zpUW_c3NIDUA5DMjAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 10:22:55] "POST /socket.io/?EIO=4&transport=polling&t=P